In [1]:
!pip3 install -r requirements.txt


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [8]:
import httpx
import pandas as pd


client = httpx.Client()
# base_url = "https://larek.itatmisis.ru/api"
base_url = "http://localhost:8000/api"

response = client.post(
    f"{base_url}/auth/login",
    data={"username": "test@test.com", "password": "Test123456"},
)
access_token = response.json()["access_token"]
header = {"Authorization": f"Bearer {access_token}"}
df = pd.read_excel("Профессии_ОКСО_ ЛЦТ_Профилум.xlsx")


for index, row in df.iterrows():
    data = {
        "name": row["Название профессии"],
        "description": row["Описание профессии для 8-9 класса"],
        "source": "Профилум",
    }
    response = client.post(f"{base_url}/profession/create", json=data, headers=header)
    if response.status_code == 200:
        print(f'Profession created for {row["Название профессии"],}')
    else:
        print(response)
        print(f'Failed to create profession for {row["Название профессии"],}')

<Response [409 Conflict]>
Failed to create profession for ('Агроном',)
<Response [409 Conflict]>
Failed to create profession for ('Маркшейдер',)
<Response [409 Conflict]>
Failed to create profession for ('Реставратор',)
<Response [409 Conflict]>
Failed to create profession for ('Слесарь по ремонту строительных машин',)
<Response [409 Conflict]>
Failed to create profession for ('Монтажник санитарно-технических систем и оборудования',)
Profession created for ('Военнослужащий',)
Profession created for ('Бизнес-аналитик',)
Profession created for ('Маркетолог',)
Profession created for ('Менеджер по продажам',)
Profession created for ('Digital-маркетолог',)
Profession created for ('Контент-менеджер',)
Profession created for ('Медсестра | Медбрат',)
Profession created for ('Врач',)
Profession created for ('Реабилитолог',)
Profession created for ('Фельдшер',)
Profession created for ('AI-тренер',)
Profession created for ('Архитектор интеллектуальных систем управления беспилотного транспорта',)


In [27]:
test_response =get_courses([        "typeManagement",
        "typeFinance",
        "typeSport",
        "typePO",
        "typeAnalytics",
    ])

In [30]:
items = test_response["items"]
len(items)

28

In [6]:
test_response_no_filters = 

In [5]:
import typing as tp
import httpx

course_url = "https://public.sravni.ru/v1/education/products"

LearningType = tp.Literal[
    "typeProgramming",
    "typeLanguage",
    "typeDesign",
    "typeSoftSkills",
    "typeCreativity",
    "typeAdmin",
    "typeManagement",
    "typeFinance",
    "typeSport",
    "typePO",
    "typeAnalytics",
]


def get_courses(
    learning_type: list[LearningType], limit: int = 28, offset: int = 28
) -> dict:
    response = httpx.post(
        course_url,
        data={"learningtype": learning_type, "limit": limit, "offset": offset},
    )
    return response.json()


items_no_filter = get_courses([], limit=200)["items"]
len(items_no_filter)

200

In [6]:
import typing as tp
import datetime as dt
from pydantic import BaseModel, Field


class Course(BaseModel):
    id: str
    name: str
    link: str
    learningtype: list[str]
    course_image: str = Field(..., alias="courseImage")
    price_all: int = Field(..., alias="priceAll")
    price: float
    time_installment: tp.Optional[int] = Field(..., alias="timeInstallment")
    date_start: tp.Optional[dt.datetime] = Field(..., alias="dateStart")
    portfolioText: tp.Optional[str]

    def to_csv(self) -> str:
        
        name = self.name.replace(';', '\\;')
        link = self.link.replace(';', '\\;')
        learningtype = ','.join(self.learningtype).replace(';', '\\;')
        course_image = self.course_image.replace(';', '\\;')
        portfolioText = self.portfolioText.replace(';', '\\;') if self.portfolioText else ''

        return f"{self.id};{name};{link};{learningtype};{course_image};{self.price_all};{self.price};{self.time_installment};{self.date_start};{portfolioText}"
    
            

In [7]:
api_data = []
for item in items_no_filter:
    try:
        api_data.append(Course.model_validate(item))
    except Exception as e:
        print(e, item,sep="\n\n")

In [8]:
with open("courses.csv", "w") as f:
    f.write("id;name;link;learningtype;course_image;price_all;price;time_installment;date_start;portfolioText\n")
    for course in api_data:
        f.write(course.to_csv() + "\n")

In [42]:
item = items_no_filter[0]
for k, v in item.items():
    print(k, v)

id 618cc443f96ff50f3596a2f8
discount {'promoCode': 'SRAVNI', 'promoCodeType': 'bestPrice', 'percentType': 'max', 'percent': 5}
name QA-инженер на Java
organization 61769cd0ae56ab001d91f896
link https://skillfactory.ru/java-qa-engineer-testirovshik-po
learningtype ['typeProgramming']
courseLoad None
isCourseLoadApproximately True
courseImage https://s91588.cdn.ngenix.net/learning-courses/618cc443f96ff50f3596a2f8/lr_4HmBzQm1oSbgxFQ-mN.png
priceAll 125070
price 113700
priceInstallment 3158
timeInstallment 36
timeAllHour None
timeAllDay None
timeAllMonth 10
dateStart 2024-01-10T00:00:00.000Z
timeStart None
stopLearning True
pause True
pauseConditions Возможность продолжить обучение на следующим потоке или заморозить обучение.
jobHelp True
jobGarant True
homeWork True
portfolioText Автотесты на Java
diploma True
diplomaContent Диплом о профессиональной переподготовке / сертификат
mentor True
knowledgeAccess True
dictionaryLevelFilterNew ['levelJuniorNew']
dictionaryGraphicFilterNew ['course

In [ ]:
{
    "id": "618cc443f96ff50f3596a2f8",
    "name": "QA-инженер на Java",
    "link": "https://skillfactory.ru/java-qa-engineer-testirovshik-po",
    "learningtype": ["typeProgramming"],
    "courseImage": "typeProgramming",
    "priceAll": 125070,
    "price": 113700,
    "timeInstallment": 36,
    "dateStart": "2024-01-10T00:00:00",
    "portfolioText": "Автотесты на Java",
    "source": "https://www.sravni.ru/kursy",
    "professionsNames": ["Разработчик баз данных"],
}

In [10]:
import torch
from transformers import AutoTokenizer, AutoModel
from typing import List
import re

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
sbert = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sbert = sbert.to(device)
sbert = sbert.eval()

emoji_pattern = re.compile("[\U00010000-\U0010ffff]", flags=re.UNICODE)


def clean(x):
    x = re.sub("<br>", " ", x)
    x = re.sub("\n", " ", x)
    x = emoji_pattern.sub("", x)
    x = re.sub(r"[abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ]", r" ", x)
    x = re.sub(r"(http\S+)|(www\S+)|([\w\d]+www\S+)|([\w\d]+http\S+)", r" ", x)
    return x


def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[
        0
    ]  # First element of model_output contains all token embeddings
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


def vectorize(text: str) -> List[float]:
    text = clean(text)
    encoded_input = tokenizer(
        text, padding=True, truncation=True, max_length=256, return_tensors="pt"
    )
    with torch.no_grad():
        model_output = sbert(**encoded_input)
    sentence_embeddings = mean_pooling(model_output, encoded_input["attention_mask"])

    return sentence_embeddings.tolist()[0]

/Users/egortarasov/uni/Хакатоны/lct-2023-yakutia/backend/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
import pandas as pd
import torch
from sklearn.metrics.pairwise import cosine_similarity




def get_classes(
    path_to_csv: str, professions: list[dict[str, str | int]], threshold: float = 0.55
):
    df = pd.read_csv(path_to_csv, on_bad_lines="skip", delimiter=";")
    df["text"] = df.apply(
        lambda x: x["name"] + ". " + x["portfolioText"]
        if type(x["portfolioText"]) != float
        else x["name"],
        axis=1,
    )
    result = []

    for course_id in df["id"]:
        row = {}
        for col_name in df[df["id"] == course_id]:
            if col_name != "text":
                row[col_name] = df[df["id"] == course_id][col_name].item()
            else:
                course_embedding = torch.Tensor(
                    [vectorize(df[df["id"] == course_id]["text"].item())]
                )
                curr_res = []
                curr_scores = []
                for profession in professions:
                    name = profession["name"]
                    description = profession["description"]
                    profession_embedding = torch.Tensor([vectorize(description)])
                    score = cosine_similarity(course_embedding, profession_embedding)
                    if score > threshold:
                        curr_res.append(name)
                        curr_scores.append(score)
                row["professionsNames"] = [
                    x for _, x in sorted(zip(curr_scores, curr_res))
                ]
        result.append(row)
    return result

In [12]:
import pandas as pd

df = pd.read_excel("Профессии_ОКСО_ ЛЦТ_Профилум.xlsx")
df.columns

Index(['№', 'Отрасль текстом', 'Название профессии',
       'Специальности ОКСО - по одной специальности может быть множетсво программ обучения, которые открывают колледжии и вузы. Специальность - то, что написано у тебя в дипломе после окончания одной из программ обучения.',
       'Уровень образования. Цифры в середине кода ОКСО хх.ХХ.хх говорят об образовательном уровне: 03 и 05 - высшее образование, 01 и 02 - среднее профессиональное образование',
       'Описание профессии для 8-9 класса'],
      dtype='object')

In [13]:
import json

with open("test_prof.json") as f:
    data = json.load(f)    
    
classes = get_classes("courses.csv", data)

In [16]:
with open("course_classification.json", "w") as course_file:
    json.dump(classes, course_file, ensure_ascii=False)

In [14]:
classes

[{'id': '618cc443f96ff50f3596a2f8',
  'name': 'QA-инженер на Java',
  'link': 'https://skillfactory.ru/java-qa-engineer-testirovshik-po',
  'learningtype': 'typeProgramming',
  'course_image': 'https://s91588.cdn.ngenix.net/learning-courses/618cc443f96ff50f3596a2f8/lr_4HmBzQm1oSbgxFQ-mN.png',
  'price_all': 125070,
  'price': 113700.0,
  'time_installment': 36.0,
  'date_start': '2024-01-10 00:00:00+00:00',
  'portfolioText': 'Автотесты на Java',
  'professionsNames': []},
 {'id': '618cc443f96ff50f3596a2f5',
  'name': 'Веб-разработчик с нуля',
  'link': 'https://skillfactory.ru/web-developer',
  'learningtype': 'typeProgramming',
  'course_image': 'https://s91588.cdn.ngenix.net/learning-courses/618cc443f96ff50f3596a2f5/2cv090TgpUjhGHYtlrvLw.png',
  'price_all': 131670,
  'price': 119700.0,
  'time_installment': 36.0,
  'date_start': '2023-12-27 00:00:00+00:00',
  'portfolioText': 'Веб-сайт на шаблонизаторе, веб-страница в Figma, интернет-магазин, SPA чат-портал, таск-менеджер, канбан-д

In [1]:
# uploading course info to server

import json
import httpx
import random
import datetime as dt

import httpx
import pandas as pd


client = httpx.Client()
# base_url = "https://larek.itatmisis.ru/api"
base_url = "http://localhost:8000/api"

response = client.post(
    f"{base_url}/auth/login",
    data={"username": "test@test.com", "password": "Test123456"},
)
print("login", response)
access_token = response.json()["access_token"]
header = {"Authorization": f"Bearer {access_token}"}

with open("course_classification.json", "r") as course_file:
    course_data = json.load(course_file)

for course in course_data:

    if isinstance(course["date_start"], float):
        course["date_start"] = dt.datetime.now() + dt.timedelta(
            weeks=random.randint(1, 10), days=random.randint(1, 25)
        )
    elif isinstance(course["date_start"], str):
        y,mo,d = map(int, course["date_start"].split(" ")[0].split("-"))
        h, m, s = map(int, course["date_start"].split(" ")[1].split("+")[0].split(":"))
        
        course["date_start"] = dt.datetime(y,mo,d,h,m,s)
    course["date_start"] = course["date_start"].isoformat()
    request_data = {
        "id": course["id"],
        "name": course["name"],
        "link": course["link"],
        "learningtype": [course["learningtype"]],
        "courseImage": course["course_image"],
        "priceAll": round(float(course["price_all"]), 2),
        "price": round(float(course["price"])),
        "timeInstallment": course["time_installment"],
        "dateStart": course["date_start"],
        "portfolioText": course["portfolioText"],
        "professionsNames": ["Разработчик беспилотных систем управления"],
        "source": "https://www.sravni.ru/",
    }

    response = httpx.post(f"{base_url}/course/create",json=request_data, headers=header)
    print(response, response.content)

login <Response [200 OK]>
<Response [200 OK]> b'{"id":"618cc443f96ff50f3596a2f8","name":"QA-\xd0\xb8\xd0\xbd\xd0\xb6\xd0\xb5\xd0\xbd\xd0\xb5\xd1\x80 \xd0\xbd\xd0\xb0 Java","link":"https://skillfactory.ru/java-qa-engineer-testirovshik-po","learning_type":["typeProgramming"],"course_image":"https://s91588.cdn.ngenix.net/learning-courses/618cc443f96ff50f3596a2f8/lr_4HmBzQm1oSbgxFQ-mN.png","price_all":125070,"price":113700.0,"time_installment":36,"date_start":"2024-01-10T00:00:00","portfolio_text":"\xd0\x90\xd0\xb2\xd1\x82\xd0\xbe\xd1\x82\xd0\xb5\xd1\x81\xd1\x82\xd1\x8b \xd0\xbd\xd0\xb0 Java","source":"https://www.sravni.ru/"}'
<Response [200 OK]> b'{"id":"618cc443f96ff50f3596a2f5","name":"\xd0\x92\xd0\xb5\xd0\xb1-\xd1\x80\xd0\xb0\xd0\xb7\xd1\x80\xd0\xb0\xd0\xb1\xd0\xbe\xd1\x82\xd1\x87\xd0\xb8\xd0\xba \xd1\x81 \xd0\xbd\xd1\x83\xd0\xbb\xd1\x8f","link":"https://skillfactory.ru/web-developer","learning_type":["typeProgramming"],"course_image":"https://s91588.cdn.ngenix.net/learning-courses/61